## Installing Dependencies

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import glob
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import shap
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Configuration & Initial Data Processing

In [ ]:
TARGET_COL = "AQI"
FORECAST_HORIZON = 48
LAG_HOURS = 72
ROLL_WINDOWS = [3, 6, 12, 24]

VAL_DAYS = 5
TEST_DAYS = 7

SEED = 42
np.random.seed(SEED)

# Load CSV
csv_path = "/kaggle/input/delhi-air-quality-dataset/final_dataset.csv"
print("Using:", csv_path)
df = pd.read_csv(csv_path)
df.head(5)

In [ ]:
# Construct proper datetime from Year, Month, Date columns
df["ds"] = pd.to_datetime(df["Year"].astype(str) + "-" +
                          df["Month"].astype(str) + "-" +
                          df["Date"].astype(str),
                          format="%Y-%m-%d",
                          errors="coerce")

df = df.dropna(subset=["ds"]).sort_values("ds").reset_index(drop=True)

In [ ]:
# Target column check
if TARGET_COL not in df.columns:
    for c in df.columns:
        if "pm2.5" in c.lower() or "pm25" in c.lower():
            TARGET_COL = c
            break

df[TARGET_COL] = pd.to_numeric(df[TARGET_COL], errors="coerce")
df = df.dropna(subset=[TARGET_COL])

# Dataset is already hourly — ensure sorted and clean
df = df.sort_values("ds").reset_index(drop=True)

## Exploratory Data Analysis

In [ ]:
print("\nEDA: Basic Info")
print(df.info())

print("\nEDA: First Datapoints")
print(df.head())

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df["ds"].head(200), df[TARGET_COL].head(200))
plt.title("AQI (first 200 hours)")
plt.xlabel("Time")
plt.ylabel("AQI")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df["ds"].tail(200), df[TARGET_COL].tail(200))
plt.title("AQI (last 200 hours)")
plt.xlabel("Time")
plt.ylabel("AQI")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
plt.hist(df[TARGET_COL], bins=50)
plt.title("Histogram of AQI")
plt.xlabel("AQI")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
df["hour"] = df["ds"].dt.hour
plt.figure(figsize=(10,4))
sns.boxplot(x="hour", y=TARGET_COL, data=df)
plt.title("AQI by Hour of Day")
plt.grid(True)
plt.show()

In [ ]:
# Correlation heatmap (for numeric columns)
num_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(12,8))
sns.heatmap(df[num_cols].corr(), cmap="coolwarm", annot=False)
plt.title("Correlation Heatmap")
plt.show()

## Feature Engineering

In [ ]:
df["dayofweek"] = df["ds"].dt.dayofweek
df["day"] = df["ds"].dt.day
df["month"] = df["ds"].dt.month

for lag in range(1, LAG_HOURS + 1):
    df[f"{TARGET_COL}_lag_{lag}"] = df[TARGET_COL].shift(lag)

for w in ROLL_WINDOWS:
    df[f"{TARGET_COL}_roll_{w}"] = df[TARGET_COL].rolling(w).mean()

df = df.dropna().reset_index(drop=True)

## Splitting Dataset

Dataset is split into Train, Test, Val. <br>
- **Test:** Data from the last 7 days
- **Val:** Data from previous 5 days (before the last week)
- **Train:** Everything Else

In [ ]:
latest = df["ds"].max()
val_start = latest - timedelta(days=TEST_DAYS + VAL_DAYS)
test_start = latest - timedelta(days=TEST_DAYS)

train_df = df[df["ds"] <= val_start].copy()
val_df = df[(df["ds"] > val_start) & (df["ds"] <= test_start)].copy()
test_df = df[df["ds"] > test_start].copy()

features = [c for c in df.columns if c not in ["ds", TARGET_COL]]

scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
val_df[features] = scaler.transform(val_df[features])
test_df[features] = scaler.transform(test_df[features])

X_train = train_df[features].values
y_train = train_df[TARGET_COL].values

X_val = val_df[features].values
y_val = val_df[TARGET_COL].values

## Model Training and Fine-Tuning

### Hyperparameter Tuning (Using Optuna)

In [ ]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 900),
        "max_depth": trial.suggest_int("max_depth", 4, 14),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 3.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "random_state": SEED
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Best parameters:", study.best_params)

### Final Model Training

In [ ]:
# Using Best Parameters from Hyperparameter Tuning
best_params = study.best_params
best_params.update({
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "random_state": SEED
})

model = xgb.XGBRegressor(**best_params)
model.fit(train_df[features], train_df[TARGET_COL])

### Feature Importance from XGBoost

In [ ]:
importance = model.get_booster().get_score(importance_type='gain')
fi_df = pd.DataFrame({
    "feature": list(importance.keys()),
    "importance": list(importance.values())
}).sort_values("importance", ascending=False)

print(fi_df.head(20))

plt.figure(figsize=(8,10))
sns.barplot(data=fi_df.head(10), x="importance", y="feature")
plt.title("Top 10 Important Features")
plt.grid(True)
plt.show()

## Evaluation on Test Set

In [ ]:
X_test = test_df[features].values
y_test = test_df[TARGET_COL].values

y_pred = model.predict(X_test)

epsilon = 1e-10 # Avoid division by zero by adding a very small epsilon
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / (y_test + epsilon))) * 100
accuracy = 100 - mape

print("Test MAE:", mae)
print("Test RMSE:", rmse)
print("MAPE (%):", mape)
print("Accuracy (%):", accuracy)

plt.figure(figsize=(12,4))
plt.plot(test_df["ds"], y_test, label="True")
plt.plot(test_df["ds"], y_pred, label="Predicted")
plt.title("Test Set Prediction vs Actual")
plt.legend()
plt.grid(True)
plt.show()

## Forecast for the Next 48 Hours

In [ ]:
# (recursive)
last_row = df.iloc[-1:].copy()
future_predictions = []
future_times = []

current_time = df["ds"].max()
current_row = last_row.copy()

for step in range(1, FORECAST_HORIZON + 1):
    current_time += timedelta(hours=1)
    future_times.append(current_time)

    for lag in range(LAG_HOURS, 1, -1):
        curr = f"{TARGET_COL}_lag_{lag}"
        prev = f"{TARGET_COL}_lag_{lag-1}"
        current_row[curr] = current_row[prev]

    current_row[f"{TARGET_COL}_lag_1"] = (
        future_predictions[-1] if len(future_predictions) > 0 else last_row[TARGET_COL].values[0]
    )

    for w in ROLL_WINDOWS:
        vals = [current_row[f"{TARGET_COL}_lag_{lag}"].values[0] for lag in range(1, w+1)]
        current_row[f"{TARGET_COL}_roll_{w}"] = np.mean(vals)

    current_row["hour"] = current_time.hour
    current_row["dayofweek"] = current_time.dayofweek
    current_row["day"] = current_time.day
    current_row["month"] = current_time.month

    temp = current_row[features].values
    temp = scaler.transform(temp)

    pred = model.predict(temp)[0]
    future_predictions.append(pred)

forecast_df = pd.DataFrame({
    "timestamp": future_times,
    "predicted_AQI": future_predictions
})

print("\nNext 48h forecast:")
print(forecast_df)

plt.figure(figsize=(12,4))
plt.plot(df.tail(100)["ds"], df.tail(100)[TARGET_COL], label="History")
plt.plot(future_times, future_predictions, marker="o", label="Forecast")
plt.title("Next 48-hour Forecast")
plt.grid(True)
plt.legend()
plt.show()